In [1]:
import os, os.path
import numpy as np
from time import time
import pandas as pd
import math as m
import setup_runs as sr
import importlib
import matplotlib.pyplot as plt
import itertools

In [14]:

dict_2.keys()

dict_keys(['Agriculture', 'Forestry', 'Ippu', 'Waste', 'commercial', 'electricity_generation', 'industry_and_mining', 'public', 'residential', 'transport'])

In [20]:
disp = list(dict_2["electricity_generation"][dict_2["electricity_generation"]["x_or_xl"].isin(["x", "xl"])]["parameter"])
disp.sort()
disp

['fuel_price_coal',
 'fuel_price_coal',
 'fuel_price_diesel',
 'fuel_price_diesel',
 'fuel_price_fuel_oil',
 'fuel_price_fuel_oil',
 'fuel_price_natural_gas',
 'fuel_price_natural_gas',
 'hydrology_exceedance_probability',
 'investment_cost_csp_solar',
 'investment_cost_csp_solar',
 'investment_cost_geothermal',
 'investment_cost_geothermal',
 'investment_cost_natural_gas_cc',
 'investment_cost_natural_gas_cc',
 'investment_cost_pv_solar',
 'investment_cost_pv_solar',
 'investment_cost_wind',
 'investment_cost_wind']

In [46]:

#df_attribute_master_id = pd.read_csv(sr.fp_csv_attribute_master)
#pull in parameters
df_params = pd.read_csv(sr.fp_csv_parameter_ranges)
#

dict_1 = pd.read_excel(os.path.join(sr.dir_ref, "parameter_x_xl_designation", "parameter_x-xl_designation.xlsx"), sheet_name = None)
dict_2 = pd.read_excel(os.path.join(sr.dir_ref, "parameter_x_xl_designation", "parameter_x-xl_designation_carlos.xlsx"), sheet_name = None)

keys_1 = ["Agriculture", "Forestry", "Ippu", "Waste"]
keys_2 = [x for x in dict_2.keys() if x not in keys_1]

params_xl = set({})
all_params_comp = set({})

# get the 
def get_params_from_traj(params):
    substr_max = "trajmax"
    substr_min = "trajmin"
    substr_mix = "trajmix"
    substr_group = "trajgroup"
    
    checkr = [x for x in params if substr_mix in x]
    dict_out = {}
    dict_out_rev = {}
    set_out = set({})
    for param in checkr:
        if "trajgroup" in param:
            s1_rep = param.split("-")[0]
            x = param.replace(s1_rep + "-", "").replace(substr_mix + "_", "")
        else:
            x = param.replace(substr_mix + "_", "")
            
        if x not in dict_out_rev.keys():
            dict_out_rev.update({x: set({param})})
        else:
            dict_out_rev.update({x: (set({param}) | dict_out_rev[x])})
        dict_out.update({param: x})
        set_out = set_out | set({x})

    return {"dict": dict_out, "dict_rev": dict_out_rev, "set": set_out}
    
    
for k in dict_1.keys():
    if k in keys_1:
        params_xl = params_xl | set(dict_1[k][dict_1[k]["x_or_xl"] != "x"]["parameter"])

        #all parameters accounted for in df_params
        all_params_comp = all_params_comp | set(dict_1[k]["parameter"])
    else:
        params_xl = params_xl | set(dict_2[k][dict_2[k]["x_or_xl"] != "x"]["parameter"])

        #all parameters accounted for in df_params
        all_params_comp = all_params_comp | set(dict_1[k]["parameter"])

#parameters that need to be added
params_add = get_params_from_traj(list(df_params["parameter"].unique()))["set"] - all_params_comp


##  SOME FUNCTIONS

def check_param(param):
    if param in params_xl:
        out = "xl"
    else:
        out = "x"
        
    #check trajgroup
    if ("trajgroup_" in param) & ("-lhs" in param):
        out = "xl"
        
    return out

def disp_str(str_in):
    str_out = str_in.replace("_", " ").replace("-", " ")
    str_out = str_out.split(" ")
    
    s_new = ""
    for s in str_out:
        if len(s) > 1:
            s = s[0].upper() + s[1:len(s)].lower()
        else:
            s = s.upper()
        s_new = s_new + " " + s
    s_new = s_new[1:len(s_new)]
    
    return s_new



##  INITIALIZE ATTRIBUTE

fields_merge = ["parameter", "sector"]
st_tuples = [tuple(x) for x in np.array(df_params[["strategy_id", "time_series_id"]].drop_duplicates())]

#initialize the parameter attribute
df_attr_pr = df_params[fields_merge].drop_duplicates()
#grab fields with "trajmax/trajmin/trajmix"

#get string identifier
substr_max = "trajmax"
substr_min = "trajmin"
substr_mix = "trajmix"
substr_group = "trajgroup"
#get parameters that should be mixed
params_mix_loop_0 = [x for x in list(df_attr_pr["parameter"]) if ((substr_mix + "_") in x)]
#get dictionary to map to sector
dict_pml_sector = {}

df_pml_append = []

for pml in params_mix_loop_0:
    #get the sector
    sec = str(df_attr_pr[df_attr_pr["parameter"] == pml]["sector"].iloc[0])
    
    #get all parameters to drop
    params_pml_drop = [pml, pml.replace(substr_mix, substr_max), pml.replace(substr_mix, substr_min)]
    df_attr_pr = df_attr_pr[~df_attr_pr["parameter"].isin(params_pml_drop)]
     
    #check if this parameter needs to be added to xl list
    pml_add_pxl = pml.replace(substr_mix + "_", "")
    if pml_add_pxl in params_xl:
        params_xl = params_xl | set({pml_add_pxl})
    #check if group is specified
    if not substr_group in pml:
        #append to the new dataframe
        df_pml_append.append([pml, sec])

df_pml_append = pd.DataFrame(df_pml_append, columns = fields_merge)



#build new attribute table
df_attr_pr = pd.concat([df_attr_pr[~df_attr_pr["parameter"].isin(df_pml_append["parameter"])], df_pml_append[list(df_attr_pr.columns)]], axis = 0)
#add in "x/xl" designation
df_attr_pr["x_xl_designation"] = [check_param(x) for x in df_attr_pr["parameter"]]
#reorder
df_attr_pr = df_attr_pr.sort_values(by = ["sector", "parameter"]).reset_index(drop = True)
#add display string
df_attr_pr["display"] = [disp_str(x) for x in list(df_attr_pr["parameter"])]

#map of replacements for selected parameters:
dict_repl_disp = {
    "Trajgroup 1 Lhs": "Copper Heat Energy - % Towards Optimistic",
    "Trajgroup 2 Lhs": "Open Pit Mine Energy - % Towards Optimistic",
    "Trajgroup 3 Lhs": "Other Industries Heat Energy - % Towards Optimistic",
    "Trajgroup 4 Lhs": "Residential Apartment ACS Energy - % Towards Optimistic",
    "Trajgroup 5 Lhs": "Residential House ACS Energy - % Towards Optimistic",
    "Trajgroup 6 Lhs": "Private Transportation Electrification - % Towards Optimistic",
    "Trajgroup 7 Lhs": "Hydrogen Adoption in Trucks - % Towards Optimistic"
}

#add in trajmix displays
dict_add = dict([[x, x.replace("Trajmix ", "").replace("Pib", "PIB")] for x in df_attr_pr["display"] if "Trajmix" in x])

for k in dict_add.keys():
    dict_repl_disp.update({k: dict_add[k]})
    
df_attr_pr["display"] = df_attr_pr["display"].replace(dict_repl_disp)

df_attr_pr.to_csv(sr.fp_csv_attribute_parameters, index = None, encoding = "UTF-8")





In [44]:
k = "industry_and_mining"
if k in keys_1:
    px = set(dict_1[k][dict_1[k]["x_or_xl"] != "x"]["parameter"])
else:
    px = set(dict_2[k][dict_2[k]["x_or_xl"] != "x"]["parameter"])

In [47]:
params_add

{'copper_heat_diesel',
 'copper_heat_solar',
 'copper_open_pit_mine_diesel',
 'copper_open_pit_mine_electricitiy',
 'other_industries_heat_coal',
 'other_industries_heat_diesel',
 'other_industries_heat_fuel_oil',
 'other_industries_heat_natural_gas',
 'other_industries_heat_pliqgas',
 'other_industries_heat_solar',
 'pib',
 'poblacion',
 'residential_apartment_acs_electric',
 'residential_apartment_acs_natural_gas',
 'residential_apartment_acs_pliqgas',
 'residential_apartment_acs_solar',
 'residential_house_acs_electric',
 'residential_house_acs_natural_gas',
 'residential_house_acs_pliqgas',
 'residential_house_acs_solar',
 'residential_retrofit_house',
 'transport_frac_aviation_hydrogen',
 'transport_frac_aviation_kerosene',
 'transport_frac_private_electric',
 'transport_frac_private_gasoline',
 'transport_frac_private_hyb',
 'transport_frac_truck_diesel',
 'transport_frac_truck_hydrogen'}

In [121]:
df_params = pd.read_csv(sr.fp_csv_parameter_ranges.replace(".csv", "_design_id-1.csv"))

arch130 = "Chile_sector_package_2021_01_30_design_id-1"
df_params_130 = sr.get_archive_run(sr.fp_csv_parameter_ranges, arch130)
#check for params that were added recently
params_newadds = set(df_params["parameter"]) - set(df_params_130["parameter"])
#check for xls that are associate with trajmix/trajgroup
params_cur = list(df_params["parameter"].unique())
param_trajs = [x for x in params_cur if substr_mix in x]
traj_xls = [x for x in params_cur if ("trajgroup" in x) and ("-lhs" in x)]

for p in list(params_xl):
    check_str = substr_mix + "_" + p
    check_list = [x for x in param_trajs if check_str in x]
    
    if len(check_list) > 0:
        pml = check_list[0]
        traj_xls = traj_xls + [pml, pml.replace(substr_mix, substr_max), pml.replace(substr_mix, substr_min)]

traj_xls = set(np.array(traj_xls).flatten())
set_params_ones = params_xl | traj_xls 
#some other parameters to add in
set_ag = [x for x in list(df_params[df_params["sector"] == "Analytica_agriculture"]["parameter"].unique()) if ("medida" in x) or ("manejo" in x)]
set_fo = [x for x in list(df_params[df_params["sector"] == "Analytica_forestry"]["parameter"].unique()) if ("medida" in x) or ("manejo" in x)]
#update set
set_params_ones = set_params_ones | set(set_ag)
set_params_ones = set_params_ones | set(set_fo)
set_params_ones = set_params_ones | set({"fp_compost", "trajmix_residential_retrofit_house"})

#loop over rows to replace with ones
new_array = np.array(df_params[["min_2050", "max_2050"]].copy())

for i in range(len(df_params)):
    p = str(df_params["parameter"].iloc[i])
    
    
    if p in set_params_ones:
        new_array[i, 0] = 1.0
        new_array[i, 1] = 1.0
    #update
    if ("trajgroup" in p) and ("-lhs" in p):
        df_params["trajgroup_no_vary_q"].iloc[i] = 1
    if p == "trajmix_residential_retrofit_house":
        print(new_array[i])
        print(i)

df_params["trajgroup_no_vary_q"] = np.array(df_params["trajgroup_no_vary_q"]).astype(int)
df_params["trajgroup_no_vary_q"] = np.array(df_params["parameter_constant_q"]).astype(int)
df_params["min_2050"] = new_array[:,0]
df_params["max_2050"] = new_array[:,1]



df_params.to_csv(sr.fp_csv_parameter_ranges.replace(".csv", "_design_0.csv"), index = None, encoding = "UTF-8")
    

[1. 1.]
3048
[1. 1.]
3049
[1. 1.]
3050
[1. 1.]
3051


/Users/jsyme/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [125]:
set(df_params["parameter_constant_q"].unique())

{nan, 1.0}

In [123]:
df_params[df_params["parameter"] == "trajgroup_8-lhs"]

,sector,time_series_id,strategy_id,type,parameter,normalize_group,trajgroup_no_vary_q,parameter_constant_q,min_2050,max_2050,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
2336,industry_and_mining,0,0,incertidumbre,trajgroup_8-lhs,NaN,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2337,industry_and_mining,0,1,incertidumbre,trajgroup_8-lhs,NaN,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2338,industry_and_mining,1,0,incertidumbre,trajgroup_8-lhs,NaN,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2339,industry_and_mining,1,1,incertidumbre,trajgroup_8-lhs,NaN,1,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
